In [ ]:
class Data:
    '''
    Esta clase tiene todos los datos necesarios para entrenar a la PINN. Los separamos en las distintas funciones de pérdida consideradas.  
    Primero la data real, luego los BC, los colloc points, y por ultimo la condiciones iniciales.   
    '''
    def __init__(self,obs_data=None,colloc_data=None,BC_data=None,IC_data=None) -> None:
        self.obs_data=obs_data
        self.colloc_data=colloc_data
        self.BC_data=BC_data
        self.IC_data=IC_data  





class DataLoader_1:
    def __init__(self,path,E,load_step) -> None:
        self.path=path
    
    #vamos a hacer una función para cargar todos los datos de una   
    def load_data(folder,E,load_stage=10):
        
        # carga las posiciones de los nodos    
        ititialPosition_data_path = "./data/ARTURO_TEST_1/NODES.txt"

        with open(ititialPosition_data_path,"r") as f:
            aux_list=f.readlines()

        ititialPosition_data=[i for i in aux_list if keep_line(i)]
        ititialPosition_data=[i.strip("\n") for i in ititialPosition_data]
        ititialPosition_data=[i.split() for i in ititialPosition_data]
        ititialPosition_data_np=np.array(ititialPosition_data).astype(float)[:,[0,1,2,3]]
        ititialPosition_data_pd=pd.DataFrame(ititialPosition_data_np,columns=["Node","X","Y","Z"]).set_index("Node")

        #carga el streess
        stress_data_path = f"./data/{folder}/PSOL_{load_stage}_NODAL_STRESSES_E{E}.txt"

        with open(stress_data_path,"r") as f:
            aux_list=f.readlines()


        stress_data=[i for i in aux_list if keep_line(i)]
        stress_data=[i.strip("\n").strip() for i in stress_data]
        patron = "[-.\d]+E-*\d{3}|^\d{0,4}|0\.0000"

        stress_data=[re.findall(patron,i) for i in stress_data]
        stress_data=np.array(stress_data,dtype=float)
        stress_data=pd.DataFrame(stress_data,columns=["Node","SX","SY","SZ","SXY","SYZ","SXZ"]).set_index("Node")

        # cargar datos de desplazamiento  
        displacement_data_path = f"./data/{folder}/PSOL_{load_stage}_NODAL_DISP_E{E}.txt"

        with open(displacement_data_path,"r") as f:
            aux_list=f.readlines()


        displacement_data=[i for i in aux_list if keep_line(i)]
        displacement_data=[i.strip("\n").strip() for i in displacement_data]
        patron = r"[-.\d]+E-*\d{3}|^\d{0,4}|-?\d+\.\d+|0\.0000"

        displacement_data=[re.findall(patron,i) for i in displacement_data]
        displacement_data=np.array(displacement_data,dtype=float)
        displacement_data=pd.DataFrame(displacement_data,columns=["Node","UX","UY","UZ","USUM"]).set_index("Node")

        # cargar datos de boundaries en el movimiento
        #cargar los nodos fijos
        with open("./data/ARTURO_TEST_1/RESTRINGED_NODES.txt","r") as f:
            restricted_data=f.readlines()

        restricted_data=[i for i in restricted_data if keep_line(i)]
        restricted_data=[i.strip("\n") for i in restricted_data]
        restricted_data=[i.split() for i in restricted_data]
        restricted_data_np=np.array(restricted_data)[:,[0,1]]
        restricted_data_pd=pd.DataFrame(restricted_data_np,columns=["Node","Direccion"])
        restricted_data_pd=restricted_data_pd.groupby("Node")["Direccion"].apply(lambda x : list(x)).to_frame().sort_index()
        restricted_data_pd.index=restricted_data_pd.index.astype(int)
        restricted_data_pd=restricted_data_pd.sort_index()
        restricted_data_pd=restricted_data_pd.rename(columns={"Direccion":"Restricciones"})

        with open("./data/ARTURO_TEST_1/FORCE_ON_NODES.txt","r") as f:
            force_data=f.readlines()


        force_data=[i for i in force_data if keep_line(i)]
        force_data=[i.strip("\n") for i in force_data]
        force_data=[i.split() for i in force_data]
        force_data_np=np.array(force_data)[:,[0,1,2]]
        force_data_pd=pd.DataFrame(force_data_np,columns=["Node","Direccion_Fuerza","Fuerza"])
        force_data_pd["Fuerza"]=force_data_pd["Fuerza"].astype(float)
        force_data_pd=force_data_pd.set_index("Node")
        force_data_pd.index=force_data_pd.index.astype(int)


        #ponemos todos los datos en común usando los nodos como clave   
        final_data=ititialPosition_data_pd.merge(stress_data,left_index=True,right_index=True,how="left").merge(force_data_pd,left_index=True,right_index=True,how="left").merge(displacement_data,left_index=True,right_index=True,how="left").merge(restricted_data_pd,left_index=True,right_index=True,how="left")

        final_data["Final_X"]=final_data["X"]-final_data["UX"]
        final_data["Final_Y"]=final_data["Y"]-final_data["UY"]
        final_data["Final_Z"]=final_data["Z"]-final_data["UZ"]


        return final_data